<a href="https://colab.research.google.com/github/ayu1729/Fake_News_Classifier/blob/main/FakeNewsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

A guide throughout the code will be provided for better understanding.

In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("/content/drive/MyDrive/Deep_Learning/news_data.csv")

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
y=data["label"]

The NaN values in the data are removed as they provide no information for any prediction.

In [5]:
x=data.drop("label",axis=1)

The labels 0 or 1 are supposed to be the output y of the model signifying whether news is fake or real.

In [6]:
x.shape,y.shape

((20800, 4), (20800,))

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [15]:
tf.__version__

'2.5.0'

In [9]:
msg=x.copy()

In [12]:
msg["title"][5]

'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart'

In [14]:
msg.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Text data pre-processing is done by using nltk lirary of python.
1.Steming of words ( using a root word for similar category of words like wash for-washing,washed etc.) 
2.All the characters except the alphabetical letters which do not carry any linguistic information are substituted by a space.
3.Stopwords of english language like "is","am","are" etc are removed because the info that they convey has very less significance in deciding the news is fake or not.
 
 All this pre-processing is done to filter the text data of TITLE of news which is suficient enought for making legit predictions. 

In [26]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
relText =[]
for i in range(len(msg)):
  res = re.sub('[^a-zA-Z]', " ", str(msg['title'][i]))
  res=res.lower()
  res=res.split()
  res=[ps.stem(words) for words in res if not words in stopwords.words('english')]
  res=' '.join(res)
  relText.append(res)
  


In [27]:
relText

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [28]:
vocabulary=4800
onehot_repr=[one_hot(words,vocabulary)for words in relText]
onehot_repr

[[2689, 1802, 4023, 2444, 398, 895, 3073, 2302, 4023, 579],
 [2466, 2379, 1121, 3894, 4550, 2108, 4056],
 [392, 1861, 2869, 3222],
 [4609, 2921, 2543, 639, 1114, 2653],
 [2178, 4550, 987, 1773, 4036, 1836, 4550, 4707, 436, 2926],
 [4426,
  1536,
  4355,
  1486,
  767,
  2793,
  723,
  1574,
  742,
  2604,
  3377,
  579,
  4284,
  148,
  4056],
 [3199, 3199, 2284, 2964, 3689, 497, 2647, 2157, 584, 2617, 4699, 2191],
 [1198, 1946, 2469, 632, 2933, 1828, 11, 1349, 4775, 2596, 558],
 [2232, 980, 1939, 1520, 2793, 1636, 2378, 2848, 2682, 1075, 4775, 2596, 558],
 [2288, 557, 2782, 2945, 2606, 991, 2793, 368, 4775, 2596, 558],
 [1210, 4404, 1487, 2861, 1038, 4546, 3463, 646, 2793, 1206],
 [911, 4787, 937, 2447, 2283, 4194, 4760, 1337],
 [973, 397, 3090, 1914, 651, 4394, 276, 3922, 2733, 393, 4020],
 [639, 1045, 398, 4546, 2793, 2606],
 [1112, 4029, 3036, 1105, 921, 1342, 1925, 2427, 4080],
 [150, 1581, 3618, 4087, 2049, 1280, 110, 4775, 2596, 558],
 [1065, 4349, 1121, 2356, 4495, 4775, 2596, 

In [30]:
max_len=20
embedded_title=pad_sequences(onehot_repr,padding='pre',maxlen=max_len)

In [31]:
embedded_title[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 2466, 2379, 1121, 3894, 4550, 2108, 4056], dtype=int32)

In [35]:

feature_no=40
model=Sequential()
model.add(Embedding(vocabulary,feature_no,input_length=max_len))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            192000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 305,001
Trainable params: 305,001
Non-trainable params: 0
_________________________________________________________________
None


One hot representation is representation which consists of the index of the word in vocabulary as its representation.

It is then converted into embedding which has used WordToVec which consists of features which have sematic importance.

These vector representation of word using features is fed into Bidirectional LSTM layer.(explanation in readme file of github)

Then output from lstm layer is  converted to single output by using a dense layer with sigmoid activation as it is a binary classification problem.

The model is at last trained with batch size of 64 and 15 epochs and gave seemingly good results with 91 % accuracy on validation set.

In [36]:
import numpy as np
x_array=np.array(embedded_title)
y_array=np.array(y)

In [37]:
x_array.shape,y_array.shape

((20800, 20), (20800,))

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_array, y_array, test_size=0.33, random_state=42)

In [39]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
218/218 [==============================] - 14s 45ms/step - loss: 0.2652 - accuracy: 0.8885 - val_loss: 0.1986 - val_accuracy: 0.9218
Epoch 2/15
218/218 [==============================] - 9s 41ms/step - loss: 0.1348 - accuracy: 0.9508 - val_loss: 0.2027 - val_accuracy: 0.9238
Epoch 3/15
218/218 [==============================] - 9s 41ms/step - loss: 0.0984 - accuracy: 0.9665 - val_loss: 0.2167 - val_accuracy: 0.9222
Epoch 4/15
218/218 [==============================] - 9s 41ms/step - loss: 0.0744 - accuracy: 0.9745 - val_loss: 0.3016 - val_accuracy: 0.9113
Epoch 5/15
218/218 [==============================] - 9s 41ms/step - loss: 0.0566 - accuracy: 0.9810 - val_loss: 0.2837 - val_accuracy: 0.9177
Epoch 6/15
218/218 [==============================] - 9s 41ms/step - loss: 0.0389 - accuracy: 0.9874 - val_loss: 0.3080 - val_accuracy: 0.9146
Epoch 7/15
218/218 [==============================] - 9s 41ms/step - loss: 0.0282 - accuracy: 0.9912 - val_loss: 0.3522 - val_accuracy: 0.916

In [42]:
y_pred=(model.predict(x_test) > 0.5).astype("int32")

In [43]:
from sklearn.metrics import confusion_matrix

In [44]:
confusion_matrix(y_test,y_pred)

array([[3111,  338],
       [ 233, 3182]])

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9168123543123543